<h1 style="font-size:50px; text-align:center">DCO Detections</h1>

In [21]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [22]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [23]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"
simple_sim_folder = data_path + "simple_mw_simulation/"
extended_sim_folder = data_path + "simulation_10yr/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")
total_DCOs_in_simple_MW = np.load(data_path + "total_DCO_in_simple_MW.npy")

total_DCOs_in_simple_MW_new = np.zeros_like(total_DCOs_in_MW)
total_DCOs_in_simple_MW_new[:, 0] = total_DCOs_in_simple_MW[0]
total_DCOs_in_simple_MW_new[:, 5] = total_DCOs_in_simple_MW[1]
total_DCOs_in_simple_MW = total_DCOs_in_simple_MW_new

In [30]:
simple_mw_fiducial = detections_4yr[:, 0].mean(axis=1) / total_DCOs_in_MW[:, 0] * total_DCOs_in_simple_MW[:, 0]
simple_mw_katie = detections_4yr[:, 5].mean(axis=1) / total_DCOs_in_MW[:, 5] * total_DCOs_in_simple_MW[:, 5]

# Calculate number of detections

In [33]:
def get_detections(dco_types=dco_types, variation_list=range(len(variations)), folder=sim_folder, totals=total_DCOs_in_MW, extended_mission=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))

    for d in range(len(dco_types)):
        for v in variation_list:
            # open the proper output file
            fname = folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            
            if extended_mission:
                fname = folder + "{}_{}_10yr_all.h5".format(dco_types[d], variations[v]["file"])
            
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    parameter = "n_detect"
                    n_detect = f["simulation"].attrs[parameter].astype(np.int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_detect))
                
                checks = np.zeros(len(n_detect))
                for i in range(len(n_detect)):
                    weights = full_data["weight"][cursor:cursor + n_detect[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights) / total_mw_weight[i] * totals[d][v]
                    cursor += n_detect[i]

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
                
            print(d, v, detections[d][v].mean())
    
    return detections

In [56]:
detections_simple = get_detections(folder=simple_sim_folder + "simple_mw_", variation_list=[0, 5], totals=total_DCOs_in_simple_MW)

0 0 52.3751446651769
0 5 96.72893783753781
1 0 24.93725218656574
1 5 101.39867208989773
2 0 17.374324879667842
2 5 43.457652745656496


In [57]:
def formation_ratio(model):
    return (total_DCOs_in_MW[:, model] /  total_DCOs_in_MW[:, 0]).round(2)

In [66]:
def fraction_ratio(model):
    return ((detections_4yr[:, model].mean(axis=1) / total_DCOs_in_MW[:, model]) / (detections_4yr[:, 0].mean(axis=1) / total_DCOs_in_MW[:, 0])).round(1)

In [70]:
for i in range(6, 10):
    print(variations[i]["file"], formation_ratio(i), fraction_ratio(i), (detections_4yr[:, i].mean(axis=1) / detections_4yr[:, 0].mean(axis=1)).round(2))

alpha0_1 [0.4  0.04 1.06] [0.9 1.1 1.2] [0.38 0.05 1.28]
alpha0_5 [0.8  0.52 0.57] [1.  1.  1.3] [0.79 0.51 0.74]
alpha2_0 [0.87 0.81 2.96] [1.1 1.1 1.2] [0.91 0.9  3.48]
alpha10 [0.35 0.33 2.41] [1.  1.2 1.8] [0.36 0.39 4.39]


In [34]:
detections_4yr = get_detections()

0 0 74.0329532213235
0 1 68.8149290199246
0 2 46.99094453972454
0 3 46.91890587905573
0 4 69.26099797363257
0 5 154.3290318193001
0 6 27.93464327369012
0 7 58.2729097140852
0 8 67.6154535109307
0 9 26.70402162619144
0 10 151.51430066855596
0 11 50.389973861134926
0 12 96.21717718968601
0 13 58.252718934233485
0 14 75.33006261623602
0 15 82.66221097394889
0 16 91.84220949516094
0 17 90.83676815018931
0 18 75.67522813889646
0 19 5.708585801823219


<ipython-input-33-c29233f4d354>:27: RuntimeWarning: invalid value encountered in double_scalars
  checks = np.sum(weights)**2 / np.sum(weights**2)


1 0 42.35845607368241
1 1 22.404356730637712
1 2 8.255220459339595
1 3 7.426608930563379
1 4 7.25588173138878
1 5 148.35334375131637
1 6 2.1124608250568686
1 7 21.754622932183523
1 8 37.96281102288457
1 9 16.309998128093063
1 10 56.81392835161141
1 11 70.23141789580879
1 12 30.105679754325177
1 13 51.90679829236058
1 14 43.355132494925826
1 15 86.57157445373693
1 16 142.87203884283383
1 17 132.14702177988704
1 18 34.0127559258105
1 19 15.529065183992527
2 0 7.9437775048188515
2 1 2.872357175852811
2 2 3.941399857096911
2 3 12.698251280785508
2 4 0.22172825228413504
2 5 18.229603118571266
2 6 10.186644511593554
2 7 5.88380652769224
2 8 27.674711660410907
2 9 34.91083185822268
2 10 9.922384582165915
2 11 6.651565800822859
2 12 7.174457270668815
2 13 8.241894157598301
2 14 7.977803612771134
2 15 15.408928837510228
2 16 33.57742069286192
2 17 7.247063984894998
2 18 8.78340758800099
2 19 8.495649460585017


In [35]:
detections_10yr = get_detections(dco_types, extended_mission=True, folder=extended_sim_folder)

0 0 117.85307042719589
0 1 107.81793943653936
0 2 75.84611789279842
0 3 71.23404612765857
0 4 109.29265007499845
0 5 239.51524701306107
0 6 43.90135517568294
0 7 91.59321622234809
0 8 109.63974114265565
0 9 42.36927988724541
0 10 229.65512412136198
0 11 76.6123071404033
0 12 153.69836509852482
0 13 91.56249372768977
0 14 120.51926634201887
0 15 130.1057040790012
0 16 142.94722094700487
0 17 142.33602763085602
0 18 112.41014867373673
0 19 9.248234384629864
1 0 71.30033554572967
1 1 36.870576113373566
1 2 13.603646703439113
1 3 12.126606619013852
1 4 11.759682194353912
1 5 216.8213949117562
1 6 3.497899772869856
1 7 34.86422965855323
1 8 62.73774394981621
1 9 26.572494024089305
1 10 96.05626501827614
1 11 117.43003777716703
1 12 49.89544216492731
1 13 84.81048474851073
1 14 72.30097755311093
1 15 145.0774172322893
1 16 229.021412431313
1 17 218.60592123378953
1 18 55.80113722951058
1 19 26.18030942350576
2 0 13.039392426318459
2 1 4.647382960910449
2 2 6.444029137268982
2 3 21.1163630846

In [37]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)

# What if we assumed everything was circular?

In [18]:
fid_sources = [None, None, None]
total_mw_weights = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_fiducial_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
        total_mw_weights[i] = f["simulation"].attrs["total_MW_weight"].sum()
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]

In [20]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr()
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr()
    
    full_mask = snr_circ > 7
    
    print(np.sum(new_source.weight[full_mask]) / np.sum(new_source.weight))
    
    print(np.sum(new_source.weight[full_mask]) / total_mw_weights[i] * total_DCOs_in_MW[i][0])

0.7507807027945674
55.584457522400825
0.8945025757444555
37.88796901335333
0.800666948268491
6.360363999609783


# Sesana specific

In [55]:
fid_sources = [None, None, None]
total_mw_weights = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_massTransferEfficiencyFixed_0_5_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
        total_mw_weights[i] = f["simulation"].attrs["total_MW_weight"].sum()
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]
    fid_sources[i].zams_mask = np.logical_and(data["m_1_ZAMS"] >= 20, data["m_1_ZAMS"] <= 120)

In [56]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr()
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr()
    
    full_mask = np.logical_and(fid_sources[i].zams_mask, snr_circ > 7)
    
    print(np.sum(new_source.weight[full_mask]) / total_mw_weights[i] * total_DCOs_in_MW[i][2])

27.72703093695445
6.985324518363619
0.06933292048296986
